In [3]:
import pandas as pd
import numpy as np
import ast

In [4]:
pd.set_option('display.max_columns', 50)

# 1. Extract

In [5]:
# Leemos los archivos respectivos
credits_df = pd.read_csv("datasets/credits.csv")
movies_df = pd.read_csv("datasets/movies_dataset.csv")

C:\Users\Une\AppData\Local\Temp\ipykernel_6124\892356293.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv("datasets/movies_dataset.csv")


# 2. Transform

## Tratamiento de la tabla MOVIES_DATASET

In [6]:
# tomamos una muestra del dataset para probar transformaciones
movies_df_short = movies_df.head(50)

In [7]:
# Examinamos las columnas y queremos detectar las estructuras anidadas.
movies_df_short.head()
#### Lista de conversiones necesarias para estructuras anidadas ######
# belongs_to_collection : dict
# genres : list of dicts
# production_companies: list of dicts
# production_contries: list of dicts
# spoken_languages: list of dicts

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


### Creamos estrategias para la conversión de estructuras anidadas

#### Estrategias Listas

In [9]:
# Definimos la siguiente función para interpretar formatos de texto
def text_interpret(text):
    """
    Amplía ast.literal_eval para manejar errores.
    Útil cuando queremos que ignore errores de conversión retornando el mismo valor.
    """
    try:
        return ast.literal_eval(text)
    except:
        return text

In [10]:
# Ya que tenemos varios casos, vamos a crear una funcion para automatizar.
# definimos una lista con los nombres de las columnas


def masive_explode(data: pd.DataFrame, lista: list):
    data_copy = data.copy()
    for column in lista:
        data_copy[column] = data_copy[column].apply(text_interpret) # podemos intentar usar text_interpret
        data_copy = data_copy.explode(column)
        data_copy.reset_index(drop=True, inplace=True)
    return data_copy


In [11]:
# Definimos las columnas a expandir y expandimos
columns = ["genres", "production_companies", "production_countries", "spoken_languages"]
movies_df_short_exploded = masive_explode(movies_df_short, columns)

In [12]:
# exploramos el dataset expandido
print(movies_df_short_exploded.shape)
movies_df_short_exploded.head(2)

(1083, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"{'id': 16, 'name': 'Animation'}",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"{'name': 'Pixar Animation Studios', 'id': 3}","{'iso_3166_1': 'US', 'name': 'United States of...",1995-10-30,373554033.0,81.0,"{'iso_639_1': 'en', 'name': 'English'}",Released,NaN,Toy Story,False,7.7,5415.0
1,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"{'id': 35, 'name': 'Comedy'}",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"{'name': 'Pixar Animation Studios', 'id': 3}","{'iso_3166_1': 'US', 'name': 'United States of...",1995-10-30,373554033.0,81.0,"{'iso_639_1': 'en', 'name': 'English'}",Released,NaN,Toy Story,False,7.7,5415.0


#### Estrategias Diccionarios

In [13]:
# Definimos una funcion para realizar la extraccion:

def masive_normalize(data: pd.DataFrame, column_list: list):
    """Ayuda a expander de forma rapida datos en forma de dicionario de diversas columnas.
    Retorna una copia del dataset provisto."""
    dataframe = data.copy()
    for outer_column in column_list:
        # expandimos columna por columna. se guarda en expanded_column
        expanded_column = pd.json_normalize(dataframe[outer_column].apply(text_interpret))
        for inner_column in expanded_column.columns:
            expanded_column.rename(columns={inner_column: outer_column+"."+inner_column}, inplace=True)
        dataframe = pd.concat([dataframe, expanded_column], axis=1)
        dataframe.drop(columns=outer_column, inplace=True)
    return dataframe
        



In [14]:

# Ejemplo conversion de diccionarios

# expandimos los diccionarios:
collecctions_expanded = pd.json_normalize(movies_df_short["belongs_to_collection"].apply(text_interpret))
# renombramos las columnas para mejor interpretabilidad:
for column in collecctions_expanded.columns:
    collecctions_expanded.rename(columns={column: "collection."+column}, inplace=True)

In [15]:
# anexamos el diccionario expandido y dropeamos la columna usada
movies_df_short = pd.concat([movies_df_short, collecctions_expanded], axis=1)
movies_df_short.drop(columns="belongs_to_collection", inplace=True)

In [16]:
# visualizacion
movies_df_short.head(2)

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection.id,collection.name,collection.poster_path,collection.backdrop_path
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,NaN,NaN,NaN


In [17]:
# Aplicamos la funcion masiva
# 1. definimos la lista de columnas
expansion_list = ["belongs_to_collection", "genres", "production_companies", "production_countries", "spoken_languages"]

# 2. aplicamos la funcion de expansion.
movies_df_short_exploded_normalized = masive_normalize(movies_df_short_exploded, expansion_list)

In [18]:
# visualizamos:
print(movies_df_short_exploded_normalized.shape)
movies_df_short_exploded_normalized.head(2)
# observamos un incremento en columnas de 24 a 31.

(1083, 31)


,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,belongs_to_collection.id,belongs_to_collection.name,belongs_to_collection.poster_path,belongs_to_collection.backdrop_path,genres.id,genres.name,production_companies.name,production_companies.id,production_countries.iso_3166_1,production_countries.name,spoken_languages.iso_639_1,spoken_languages.name
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,False,7.7,5415.0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,16,Animation,Pixar Animation Studios,3,US,United States of America,en,English
1,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,False,7.7,5415.0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,35,Comedy,Pixar Animation Studios,3,US,United States of America,en,English


### Aplicación de la estrategia para la tabla movies

In [53]:
#1. Expansion de listas
print(f"Previous shape: {movies_df.shape}")
explode_columns = ["genres", "production_companies", "production_countries", "spoken_languages"]
movies_df_exploded = masive_explode(movies_df, explode_columns)
print(f"New shape: {movies_df_exploded.shape}")
movies_df_exploded.head(2)

Previous shape: (45466, 24)
New shape: (419779, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"{'id': 16, 'name': 'Animation'}",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"{'name': 'Pixar Animation Studios', 'id': 3}","{'iso_3166_1': 'US', 'name': 'United States of...",1995-10-30,373554033.0,81.0,"{'iso_639_1': 'en', 'name': 'English'}",Released,NaN,Toy Story,False,7.7,5415.0
1,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"{'id': 35, 'name': 'Comedy'}",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"{'name': 'Pixar Animation Studios', 'id': 3}","{'iso_3166_1': 'US', 'name': 'United States of...",1995-10-30,373554033.0,81.0,"{'iso_639_1': 'en', 'name': 'English'}",Released,NaN,Toy Story,False,7.7,5415.0


In [54]:
#2. expansion de diccionarios
print(f"Previous shape: {movies_df_exploded.shape}")
expansion_list = ["belongs_to_collection", "genres", "production_companies", "production_countries", "spoken_languages"]
movies_df_exploded_normalized = masive_normalize(movies_df_exploded, expansion_list)
print(f"New shape: {movies_df_exploded_normalized.shape}")
movies_df_exploded_normalized.head(2)

Previous shape: (419779, 24)
New shape: (419779, 31)


,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,belongs_to_collection.id,belongs_to_collection.name,belongs_to_collection.poster_path,belongs_to_collection.backdrop_path,genres.id,genres.name,production_companies.name,production_companies.id,production_countries.iso_3166_1,production_countries.name,spoken_languages.iso_639_1,spoken_languages.name
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,False,7.7,5415.0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English
1,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,False,7.7,5415.0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,35.0,Comedy,Pixar Animation Studios,3.0,US,United States of America,en,English


### Imputación de valores nulos: Columnas budget y revenue

In [55]:
movies_df_exploded_normalized["revenue"] = movies_df_exploded_normalized["revenue"].fillna(0)
movies_df_exploded_normalized["budget"] = movies_df_exploded_normalized["budget"].fillna(0)

### Tratamiento fechas

In [56]:
# Se realiza cambio a formato fecha. Se aplica coerce para convertir valores erroneos a nulos y se dropean.
print(f"Previous shape: {movies_df_exploded_normalized.shape}")
movies_df_exploded_normalized["release_date"] = pd.to_datetime(movies_df_exploded_normalized["release_date"], format="%Y-%m-%d", errors="coerce")
movies_df_exploded_normalized.dropna(axis=0, subset="release_date",inplace=True)
movies_df_exploded_normalized.reset_index(drop=True, inplace=True)
print(f"New shape: {movies_df_exploded_normalized.shape}")

Previous shape: (419779, 31)
New shape: (419588, 31)


In [57]:
#Agregamos la columna "release_year" despues de "release_date".
movies_df_exploded_normalized.insert(11,"release_year", movies_df_exploded_normalized["release_date"].dt.year)

### Creacion de columna return

In [58]:
# creamos la siguiente funcion para manejar los errores de division 0
def create_return(row):
    try:
        return row["revenue"] / row["budget"]
    except:
        return 0


In [59]:
# cambiamos el tipo de data de budget a numerica
movies_df_exploded_normalized["budget"] = movies_df_exploded_normalized["budget"].astype('int32')

In [60]:
# Creamos la nueva columna return aplicando la funcion
movies_df_exploded_normalized["return"] = movies_df_exploded_normalized.apply(create_return, axis=1)

### Eliminación columnas video,imdb_id,adult,original_title,poster_path y homepage.

In [61]:
movies_df_exploded_normalized.drop(columns=["adult", "video", "imdb_id", "original_title", "poster_path", "homepage",  "belongs_to_collection.poster_path", "belongs_to_collection.backdrop_path" ], axis=1, inplace=True)

In [62]:
movies_df_exploded_normalized.shape

(419588, 25)

In [63]:
movies_df_exploded_normalized.columns
movies_df_exploded_normalized.rename(columns=
                                     {
                                        'belongs_to_collection.id' : 'collection.id',
                                        'belongs_to_collection.name': 'collection.name',
                                     }, inplace=True)

In [35]:
movies_df_exploded_normalized.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'release_year', 'revenue', 'runtime', 'status',
       'tagline', 'title', 'vote_average', 'vote_count', 'collection.id',
       'collection.name', 'genres.id', 'genres.name',
       'production_companies.name', 'production_companies.id',
       'production_countries.iso_3166_1', 'production_countries.name',
       'spoken_languages.iso_639_1', 'spoken_languages.name', 'return'],
      dtype='object')

In [64]:
#Vamos a eliminar columnas innecesarias

#runtime: no es un valor que requiramos para el análisis
# status: vamos a remover las peliculas que no hayan sido estrenadas y luego remover la columna
#collection.id: relacionado a collection.name
# genres.id: relacionado a genres.name
# production_companies.id : relacionado a production_companies.name
# spoken_languages.name asociado a spoken_languages.iso_639_1
# production_countries.name	: asociado a production_countries.iso_3166_1

mask = (movies_df_exploded_normalized["status"]=="Released")
movies_df_exploded_normalized = movies_df_exploded_normalized[mask]


column_drop=["runtime", "status", "collection.id", "genres.id", "production_companies.id",  "spoken_languages.name", "production_countries.name"]
movies_df_exploded_normalized.drop(columns=column_drop, inplace=True)

In [66]:
movies_df_exploded_normalized.shape
movies_df_exploded_normalized.to_csv("transformed/movies_df.csv", index=False)

In [65]:
movies_df_exploded_normalized.drop_duplicates().shape

(416961, 18)

### Optimización de consultas
Se crea el siguiente dataset para optimización de consultas.
- El dataset previamente obtenido se guardará para realizar el EDA y posteriormente entrenar el modelo ML.
- Se dropeará columnas no utilizadas en las consultas y se guardará el dataset en otro directorio: transformed_consultas.

In [71]:
movies_df_exploded_normalized.drop(columns=["genres.name","overview", "original_language", "tagline", "collection.name", "production_companies.name", "production_countries.iso_3166_1", 'spoken_languages.iso_639_1']).drop_duplicates().to_csv("transformed_consultas/movies_df_small.csv", index=False)

## Tratamiento de la tabla credits_df

Si revisamos las definiciones de CAST y CREW, son dos conceptos distintos. 
- CAST: Actores que actúan en la película.
- CREW: Personal técnico y creativo responsable de llevar a cabo la producción.

Talvez lo mejor en este caso sería, cambiar la disposición de los datos y despues expandir el dataset, o crear dos dataframes distintos.

In [73]:
#visualizamos la estructura de cast
ast.literal_eval(credits_df.iloc[0][0])[0]

C:\Users\Une\AppData\Local\Temp\ipykernel_6124\115940989.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ast.literal_eval(credits_df.iloc[0][0])[0]


{'cast_id': 14,
 'character': 'Woody (voice)',
 'credit_id': '52fe4284c3a36847f8024f95',
 'gender': 2,
 'id': 31,
 'name': 'Tom Hanks',
 'order': 0,
 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}

In [74]:
#visualizamos la estructura de crew
ast.literal_eval(credits_df.iloc[55][1])[0]

C:\Users\Une\AppData\Local\Temp\ipykernel_6124\2403984185.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ast.literal_eval(credits_df.iloc[55][1])[0]


{'credit_id': '57b42992c3a36875d50050d9',
 'department': 'Directing',
 'gender': 2,
 'id': 30586,
 'job': 'Director',
 'name': 'Robert Tinnell',
 'profile_path': None}

Vamos abordar de la siguiente, teniendo en cuenta la consigna del proyecto.
- Crear 2 datasets distintos, 1 de cast y otro de crew.
- La cosigna solicita solo buscar directores de peliculas para el segundo set, asi que mantendremos las entradas que pertenzecan al departamento "Directing"

### Dataframe cast

In [75]:
# Creamos un nuevo df con las columnas cast y id.
credits_df_cast = credits_df.drop(columns="crew")
# Expandimos el cast para cada id.
explosion_list = ["cast"]
credits_df_cast_exploded = masive_explode(credits_df_cast, explosion_list)
credits_df_cast_exploded.head(5)

,cast,id
0,"{'cast_id': 14, 'character': 'Woody (voice)', ...",862
1,"{'cast_id': 15, 'character': 'Buzz Lightyear (...",862
2,"{'cast_id': 16, 'character': 'Mr. Potato Head ...",862
3,"{'cast_id': 17, 'character': 'Slinky Dog (voic...",862
4,"{'cast_id': 18, 'character': 'Rex (voice)', 'c...",862


In [76]:
# Normalizamos los diccionarios.
credits_df_cast_exploded_normalized = masive_normalize(credits_df_cast_exploded, ["cast"])
credits_df_cast_exploded_normalized.head(10)

,id,cast.cast_id,cast.character,cast.credit_id,cast.gender,cast.id,cast.name,cast.order,cast.profile_path
0,862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,862,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,862,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,862,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,862,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg
5,862,19.0,Hamm (voice),52fe4284c3a36847f8024fa9,2.0,7907.0,John Ratzenberger,5.0,/yGechiKWL6TJDfVE2KPSJYqdMsY.jpg
6,862,20.0,Bo Peep (voice),52fe4284c3a36847f8024fad,1.0,8873.0,Annie Potts,6.0,/eryXT84RL41jHSJcMy4kS3u9y6w.jpg
7,862,26.0,Andy (voice),52fe4284c3a36847f8024fc1,0.0,1116442.0,John Morris,7.0,/vYGyvK4LzeaUCoNSHtsuqJUY15M.jpg
8,862,22.0,Sid (voice),52fe4284c3a36847f8024fb1,2.0,12901.0,Erik von Detten,8.0,/twnF1ZaJ1FUNUuo6xLXwcxjayBE.jpg
9,862,23.0,Mrs. Davis (voice),52fe4284c3a36847f8024fb5,1.0,12133.0,Laurie Metcalf,9.0,/unMMIT60eoBM2sN2nyR7EZ2BvvD.jpg


In [79]:
#Dropeamos columnas innecesarias
credits_df_cast_exploded_normalized.drop(columns=['cast.cast_id', 'cast.credit_id', 'cast.gender',
       'cast.id', 'cast.order', 'cast.profile_path'], inplace=True)

In [83]:
credits_df_cast_exploded_normalized.shape
credits_df_cast_exploded_normalized.drop_duplicates(inplace=True)
credits_df_cast_exploded_normalized.shape

(564348, 3)

In [ ]:
#Verificamos cantidad de valores nulos. 
print(credits_df_cast_exploded_normalized["cast.name"].isna().sum())
print(credits_df_cast_exploded_normalized["cast.character"].isna().sum())

2414
2414


In [92]:
#Podemos dropear al ser una cantidad minima
credits_df_cast_exploded_normalized.dropna(inplace=True)

In [97]:
credits_df_cast_exploded_normalized.shape

(561934, 3)

### Dataframe Crew

In [98]:
# Creamos un nuevo df con las columnas crew y id.
credits_df_crew = credits_df.drop(columns="cast")
# Expandimos el crew para cada id.
credits_df_crew_exploded = masive_explode(credits_df_crew, ["crew"])
credits_df_crew_exploded.head(5)

,crew,id
0,"{'credit_id': '52fe4284c3a36847f8024f49', 'dep...",862
1,"{'credit_id': '52fe4284c3a36847f8024f4f', 'dep...",862
2,"{'credit_id': '52fe4284c3a36847f8024f55', 'dep...",862
3,"{'credit_id': '52fe4284c3a36847f8024f5b', 'dep...",862
4,"{'credit_id': '52fe4284c3a36847f8024f61', 'dep...",862


In [99]:
# Normalizamos diccionarios
credits_df_crew_exploded_normalized = masive_normalize(credits_df_crew_exploded, ["crew"])
credits_df_crew_exploded_normalized.head(10)

,id,crew.credit_id,crew.department,crew.gender,crew.id,crew.job,crew.name,crew.profile_path
0,862,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,862,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,862,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,862,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,862,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg
5,862,52fe4284c3a36847f8024f67,Production,1.0,12894.0,Producer,Bonnie Arnold,None
6,862,52fe4284c3a36847f8024f6d,Production,0.0,12895.0,Executive Producer,Ed Catmull,None
7,862,52fe4284c3a36847f8024f73,Production,2.0,12896.0,Producer,Ralph Guggenheim,None
8,862,52fe4284c3a36847f8024f79,Production,2.0,12897.0,Executive Producer,Steve Jobs,/mOMP3SwD5qWQSR0ldCIByd3guTV.jpg
9,862,52fe4284c3a36847f8024f8b,Editing,2.0,8.0,Editor,Lee Unkrich,/bdTCCXjgOV3YyaNmLGYGOxFQMOc.jpg


In [103]:
#dropeamos columnas innecesarias
credits_df_crew_exploded_normalized = credits_df_crew_exploded_normalized.drop(columns=["crew.credit_id", "crew.gender", "crew.id", "crew.profile_path"]).drop_duplicates().dropna()
credits_df_crew_exploded_normalized.shape

(463696, 4)

# 3. Load

Vamos a guardar las tablas en un nuevo archivo

In [104]:

credits_df_crew_exploded_normalized.to_csv("transformed/credits_crew_df.csv", index=False)
credits_df_cast_exploded_normalized.to_csv("transformed/credits_cast_df.csv", index=False)
#movies_df_exploded_normalized.to_csv("transformed/movies_df.csv", index=False)